---
title: Assignment 1
format: html
---

# Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
from pandas.core.common import random_state
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_val_score, train_test_split

# Importing and Preprocessing the data

In [ ]:
df = pd.read_csv('Data/train.csv')
df.drop_duplicates(inplace=True)

print(f"The shape of the raw data is: {df.shape}")
print("-------------------------------------------")

cols_with_nulls = df.isna().sum() * 100/ df.shape[0]

cols_with_nulls = cols_with_nulls[cols_with_nulls > 0].keys()
print("Columns of the raw data with Null values: ",cols_with_nulls)

for col in cols_with_nulls:
  print('Indexs in Col', col, ' with null vals in', df.index[df[col].isna()])
  df.drop(labels=df.index[df[col].isna()], inplace=True)

print("The shape after removing the Null vals: ", df.shape)

print("-------------------------------------------")

In [ ]:
class OutlierFiltering:
  
  def __init__(self, rows, a) -> None:
    self.q1 = rows.quantile(0.25)
    self.q2 = rows.quantile(0.75)
    self.gap = self.q2 - self.q1
    self.alpha = a

  def SetRatio(self, alpha) -> None:
    self.alpha = alpha

  def FilterTheOutliers(self, val) -> bool:
    return not (val <= self.q2 + self.alpha * self.gap and val >= self.q1 - self.alpha * self.gap)

In [ ]:
remover1 = OutlierFiltering(df['Feature1'], 1.5)
remover3 = OutlierFiltering(df['Feature3'], 1.5)
remover4 = OutlierFiltering(df['Feature4'], 1.5)

# df.drop(label=df.index[df['Feature1'].map(remover1.FilterTheOutliers)], inplace=True)
df.drop(df.index[df['Feature1'].map(remover1.FilterTheOutliers)], inplace= True)
df.drop(df.index[df['Feature3'].map(remover3.FilterTheOutliers)], inplace= True)
df.drop(df.index[df['Feature4'].map(remover4.FilterTheOutliers)], inplace= True)

df.shape
print(df.keys())

In [ ]:
# Standardizing the data

scaler = StandardScaler()
X_data_scaled = scaler.fit_transform(df[['Feature1', 'Feature2', 'Feature3', 'Feature4']])
Y_data_scaled = df['Label']

In [ ]:
list_of_test_mse_linear = []
list_of_train_mse_linear = []
list_of_test_mse_lasso = []
list_of_train_mse_lasso = []
list_of_test_mse_ridge = []
list_of_train_mse_ridge = []

for i in range(1,10):
  X_train, X_test, Y_train, Y_test = train_test_split(X_data_scaled, Y_data_scaled, random_state=27, test_size=0.2)
  X_poly = PolynomialFeatures(degree=i)
  X_poly_train = X_poly.fit_transform(X_train)
  X_poly_test = X_poly.transform(X_test)
  
  poly_model = LinearRegression()
  poly_lasso = Lasso(alpha=10)
  poly_ridge = Ridge(alpha=100)
  
  poly_model.fit(X_poly_train, Y_train)
  poly_lasso.fit(X_poly_train, Y_train)
  poly_ridge.fit(X_poly_train, Y_train)

  train_pred_linear = poly_model.predict(X_poly_train)
  train_pred_lasso = poly_lasso.predict(X_poly_train)
  train_pred_ridge = poly_ridge.predict(X_poly_train)
  test_pred_linear = poly_model.predict(X_poly_test)
  test_pred_lasso = poly_lasso.predict(X_poly_test)
  test_pred_ridge = poly_ridge.predict(X_poly_test)

  mse_train_pred_linear = mean_squared_error(Y_train, train_pred_linear)
  mse_train_pred_lasso = mean_squared_error(Y_train, train_pred_lasso )
  mse_train_pred_ridge = mean_squared_error(Y_train, train_pred_ridge)
  mse_test_pred_linear = mean_squared_error(Y_test, test_pred_linear)
  mse_test_pred_lasso = mean_squared_error(Y_test, test_pred_lasso )
  mse_test_pred_ridge = mean_squared_error(Y_test, test_pred_ridge)

  list_of_train_mse_linear.append(min(mse_train_pred_linear,100000))
  list_of_train_mse_lasso.append(min(mse_train_pred_lasso,100000))
  list_of_train_mse_ridge.append(min(mse_train_pred_ridge,100000))
  list_of_test_mse_linear.append(min(mse_test_pred_linear,100000))
  list_of_test_mse_lasso.append(min(mse_test_pred_lasso, 100000))
  list_of_test_mse_ridge.append(min(mse_test_pred_ridge, 100000))

plt.plot(list_of_train_mse_linear)
plt.plot(list_of_train_mse_lasso)
plt.plot(list_of_train_mse_ridge)


In [ ]:
plt.plot(list_of_test_mse_linear, color='r')
plt.plot(list_of_test_mse_lasso, color = 'g')
plt.plot(list_of_test_mse_ridge, color = 'b')

In [ ]:
poly_model = LinearRegression()
poly_X = PolynomialFeatures(degree=2)
poly_X_data = poly_X.fit_transform(X_data_scaled)
poly_model.fit(poly_X_data, Y_data_scaled)

In [ ]:
df = pd.read_csv('Data/test.csv')
X_df = (df[['Feature1', 'Feature2', 'Feature3', 'Feature4']])
scaled_dataa = scaler.transform(X_df)
X_df = poly_X.transform(scaled_dataa)


submission_df = pd.DataFrame({'id':range(60), 'Label':range(60)})
submission_df['Label'] = poly_model.predict(X_df)

submission_df.to_csv('Data/submission.csv', index=False)